# Exploration des données de prix et squelette d'extraction

In [1]:
import pandas as pd

In [55]:
df_eauP_detail_2023 = pd.read_csv("C:/Users/Antoine/Documents/AquaVision/donnees/brutes/eau_potable/detail_AEP_2023.csv", encoding="Windows-1252", sep=";", low_memory=False)
df_eauP_tarif_2023 = pd.read_csv("C:/Users/Antoine/Documents/AquaVision/donnees/brutes/eau_potable/tarif_AEP_2023.csv", encoding="Windows-1252", sep=";", low_memory=False)

## Exploration du premier df pour le détail

In [54]:
df_eauP_detail_2023

,Id SISPEA de la collectivité,Type collectivité,Mode de gestion,Pop de l'entité de gestion sans double compte,Statut,D101.0,Verif_D101.0,CalculAuto_D101.0,D102.0,Verif_D102.0,...,Prix Tranche 7 (part délégataire),Borne inférieure Tranche 8 (part délégataire),Borne supérieure Tranche 8 (part délégataire),Prix Tranche 8 (part délégataire),Borne inférieure Tranche 9 (part délégataire),Borne supérieure Tranche 9 (part délégataire),Prix Tranche 9 (part délégataire),Borne inférieure Tranche 10 (part délégataire),Borne supérieure Tranche 10 (part délégataire),Prix Tranche 10 (part délégataire)
0,181917,Commune,Régie,788.0,Confirmé / publié,788.0,Sans anomalie apparente,0.0,"3,64",Sans anomalie apparente,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,182199,Syndicat Intercommunal à Vocation Unique,Délégation,18072.0,Confirmé / publié,17769.0,Sans anomalie apparente,0.0,"2,72",Sans anomalie apparente,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,181921,Commune,Régie,125.0,Confirmé / publié,172.0,Sans anomalie apparente,0.0,"2,12",Sans anomalie apparente,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,196926,Communauté d'agglomération,Régie,185.0,Confirmé / publié,183.0,Sans anomalie apparente,0.0,"1,4",Sans anomalie apparente,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,182064,Commune,Régie,82.0,En attente de saisie,NaN,Non vérifié,0.0,NaN,Non vérifié,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10226,218341,Établissement public territorial,Délégation,NaN,En cours de saisie,NaN,Non vérifié,0.0,"2,31",Non vérifié,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10227,97282,Syndicat Mixte,Délégation,3619122.0,Confirmé / publié,3977917.0,Sans anomalie apparente,0.0,"2,14",Sans anomalie apparente,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10228,198916,Établissement public territorial,Délégation,437145.0,Confirmé / publié,440811.0,Sans anomalie apparente,0.0,"1,38",Sans anomalie apparente,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10229,218341,Établissement public territorial,NaN,2740.0,En cours de saisie,2740.0,Non vérifié,0.0,"2,96",Non vérifié,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- **DPT du siège de la coll.**: On peut supprimer il sera signalé dans l'autre df avec la commune.
- **Id SISPEA de la collectivité** : A garder pour le merge mais il faut supprimer le "CollectivityId=" dans la valeur pour ne garder que l'id.
- **Nom collectivité** : Sur la fiche SISPEA renvoyée sur le streamlit.
- **Type collectivité** : Important pour ML.
- **Id SISPEA de l'entité de gestion** : Sur la fiche SISPEA renvoyée sur le streamlit.
- **Nom de l'entité de gestion** : Sur la fiche SISPEA renvoyée sur le streamlit.
- **Mode de gestion** : Important pour ML.
- **Pop de l'entité de gestion sans double compte** : Peut impacter le prix, on garde pour le ML.
- **Statut** : On peut supprimer les lignes "En attente de saisie" et "En cours de saisie" car les données n'y sont pas encore. 
- **[D101.0, Verif_D101.0, CalculAuto_D101.0]** : D101.0 correspond au nbre d'habitants désservis: ok pour ML. Les 2 autres sont du détail non pertinent.
- **[D102.0, Verif_D102.0, CalculAuto_D102.0]** : D102.0 est le prix au m³, donc ce qu'il va falloir deviner. Les 2 autres sont du détail non pertinent.
- **[VP.056, VP.190, VP.177, VP.216, VP.179, VP.178, VP.219, VP.215, VP.214, VP.191, VP.175, VP.213, VP.229, DC.184]** : Toutes ces colonnes sont du détail qui permettent de calculer D102.0
- **[Borne inférieure Tranche 1 (part collectivité), Borne supérieure Tranche 1 (part collectivité), Prix Tranche 1 (part collectivité)] * 10**: Toutes ces colonnes sont du détail qui permettent de calculer D102.0
- **[Borne inférieure Tranche 1 (part délégataire), Borne supérieure Tranche 1 (part délégataire), Prix Tranche 1 (part délégataire)] * 10** : Toutes ces colonnes sont du détail qui permettent de calculer D102.0

1. On se rend compte qu'il y trop de trous dans les données et qu'un Machine Learning qui déterminerait le prix de l'eau en fonction d'autres indicateurs ne serait pas pertinent 
2. On va pouvoir seulement faire une régression linéaire basée sur l'historique et donc éliminer toutes les autres donnée 
3. On garde donc seulement l'ID SISPEA de la collectivité pour le merge et le D102.0 qui est le prix

In [94]:
# Modification des valeurs de la colonn id SISPEA pour futur merge
df_eauP_detail_2023['Id SISPEA de la collectivité'] = df_eauP_detail_2023['Id SISPEA de la collectivité'].str.replace('CollectivityId=', '').astype(int)

# On ne garde donc que cette colonne et la D102.0:
df_a_merger_detail = df_eauP_detail_2023[['Id SISPEA de la collectivité', 'D102.0']]

In [95]:
df_a_merger_detail

,Id SISPEA de la collectivité,D102.0
0,181917,"3,64"
1,182199,"2,72"
2,181921,"2,12"
3,196926,"1,4"
4,182064,NaN
...,...,...
10226,218341,"2,31"
10227,97282,"2,14"
10228,198916,"1,38"
10229,218341,"2,96"


## Exploration du 2e df

In [166]:
df_eauP_tarif_2023

,DPT de la commune,Nom de la commune adhérente,Identifiant SISPEA de la commune adhérente,Code INSEE de la commune adhérente,Population de la commune,Type d'adhésion,Nom de la collectivité via laquelle la commune adhère,N° SIREN de la collectivité via laquelle la commune adhère,Identifiant SISPEA de la collectivité via laquelle la commune adhère,Nom de la collectivité de l'entité de gestion à laquelle la commune adhère,...,Zone tarifaire 1,Tarif zone tarifaire 1,Zone tarifaire 2,Tarif zone tarifaire 2,Zone tarifaire 3,Tarif zone tarifaire 3,Zone tarifaire 4,Tarif zone tarifaire 4,Zone tarifaire 5,Tarif zone tarifaire 5
0,01,Cerdon,348,01068,788.0,Adhérent direct,NaN,NaN,NaN,Cerdon,...,Indicateur tarif,"3,64",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01,Tossiat,672,01422,1390.0,Adhérent direct,NaN,NaN,NaN,SIE Ain Veyle Revermont,...,Indicateur tarif,"2,72",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01,Dompierre-sur-Veyle,418,01145,1214.0,Adhérent direct,NaN,NaN,NaN,SIE Ain Veyle Revermont,...,Indicateur tarif,"2,72",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01,La Tranclière,675,01425,291.0,Adhérent direct,NaN,NaN,NaN,SIE Ain Veyle Revermont,...,Indicateur tarif,"2,72",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01,Montagnat,515,01254,2193.0,Adhérent direct,NaN,NaN,NaN,SIE Ain Veyle Revermont,...,Indicateur tarif,"2,72",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33901,93,Bondy,36616,93010,53509.0,Adhérent direct,NaN,NaN,NaN,Est Ensemble (T8),...,Indicateur tarif,"1,38",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33902,93,Noisy-le-Sec,36634,93053,45156.0,Adhérent direct,NaN,NaN,NaN,Est Ensemble (T8),...,Indicateur tarif,"1,38",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33903,93,Pantin,36635,93055,60597.0,Adhérent direct,NaN,NaN,NaN,Est Ensemble (T8),...,Indicateur tarif,"1,38",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33904,93,Le Pré-Saint-Gervais,36638,93061,17342.0,Adhérent direct,NaN,NaN,NaN,Est Ensemble (T8),...,Indicateur tarif,"1,38",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- **Comme on ne garde que la région parisienne, une suppression des autres départements à été faite**
- Sur cette région les colonnes vides comme les **zones tarifaires supplémentaires**, ont été supprimées (aucune data donc aucune utilité)
- **N° SIREN de la collectivité** via laquelle la commune adhère : Info a retrouvé sur les autres référencement, là ça n’a pas d’importance
- **Identifiant SISPEA de la collectivité** Servira pour le merge
- **Nom de la collectivité** de l'entité de gestion à laquelle la commune adhère : Info a retrouvé ailleurs
- **Type de collectivité** : affiché ailleurs
- **Avec ou sans CCSPL** : info non importante
- **Nom de l'entité de gestion à laquelle la commune adhère** / **Identifiant SISPEA de l'entité de gestion à laquelle la commune adhère**: info a retrouvé ailleurs
- **Code UGE** (eau potable) : Pas importante


On ne gardera que le nom de la commune (d'Ile-de-France) et l'ID SISPEA de la collectivité pour avoir un df final donnant le D102.0 pour chaque commune 

In [ ]:
# Filtrage du df pour récupérer ce que l'on veut
df_a_merger_commune = df_eauP_tarif_2023[df_eauP_tarif_2023['DPT de la commune'].isin(['75', '77', '78', '91', '92', '93', '94', '95'])][["DPT de la commune", "Nom de la commune adhérente", "Identifiant SISPEA de la collectivité de l'entité de gestion à laquelle la commune adhère"]]

## Merge des 2 df:

In [167]:

df_final = pd.merge(df_a_merger_commune, df_a_merger_detail, how = 'left', left_on = "Identifiant SISPEA de la collectivité de l'entité de gestion à laquelle la commune adhère", right_on = "Id SISPEA de la collectivité").drop(["Identifiant SISPEA de la collectivité de l'entité de gestion à laquelle la commune adhère", "Id SISPEA de la collectivité"], axis = 1)
df_final

,DPT de la commune,Nom de la commune adhérente,D102.0
0,78,Dannemarie,"2,72"
1,78,Mondreville,"2,48"
2,77,Bransles,"2,28"
3,95,Ronquerolles,"3,45"
4,77,Hautefeuille,"2,37"
...,...,...,...
9885,94,Périgny,NaN
9886,94,Périgny,"2,32"
9887,94,Périgny,"2,93"
9888,94,Périgny,"2,31"


### On se rend compte que si on compare au nombre de lignes de communes Ile-de-France il y a beaucoup de doublons:

In [168]:
df_verif = df_eauP_tarif_2023[df_eauP_tarif_2023['DPT de la commune'].isin(['75', '77', '78', '91', '92', '93', '94', '95'])]
df_verif

,DPT de la commune,Nom de la commune adhérente,Identifiant SISPEA de la commune adhérente,Code INSEE de la commune adhérente,Population de la commune,Type d'adhésion,Nom de la collectivité via laquelle la commune adhère,N° SIREN de la collectivité via laquelle la commune adhère,Identifiant SISPEA de la collectivité via laquelle la commune adhère,Nom de la collectivité de l'entité de gestion à laquelle la commune adhère,...,Zone tarifaire 1,Tarif zone tarifaire 1,Zone tarifaire 2,Tarif zone tarifaire 2,Zone tarifaire 3,Tarif zone tarifaire 3,Zone tarifaire 4,Tarif zone tarifaire 4,Zone tarifaire 5,Tarif zone tarifaire 5
9415,78,Dannemarie,32438,78194,220.0,Adhérent direct,NaN,NaN,NaN,SIE RÉGION BOUTIGNY sur OPTON,...,Indicateur tarif,"2,72",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9435,78,Mondreville,32530,78413,410.0,Adhérent direct,NaN,NaN,NaN,"SIE OULINS, GILLES, MESNIL-SIMON",...,Indicateur tarif,"2,48",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15894,77,Bransles,31902,77050,539.0,Commune desservie seulement,NaN,NaN,NaN,Dordives,...,Indicateur tarif,"2,28",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21710,95,Ronquerolles,36843,95529,894.0,Adhérent direct,NaN,NaN,NaN,Syndicat intercommunal des eaux du plateau du ...,...,Indicateur tarif,"3,45",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29146,77,Hautefeuille,32069,77224,257.0,Adhérent direct,NaN,NaN,NaN,CA Coulommiers Pays de Brie (CACPB),...,Indicateur tarif,"2,37",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33901,93,Bondy,36616,93010,53509.0,Adhérent direct,NaN,NaN,NaN,Est Ensemble (T8),...,Indicateur tarif,"1,38",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33902,93,Noisy-le-Sec,36634,93053,45156.0,Adhérent direct,NaN,NaN,NaN,Est Ensemble (T8),...,Indicateur tarif,"1,38",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33903,93,Pantin,36635,93055,60597.0,Adhérent direct,NaN,NaN,NaN,Est Ensemble (T8),...,Indicateur tarif,"1,38",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33904,93,Le Pré-Saint-Gervais,36638,93061,17342.0,Adhérent direct,NaN,NaN,NaN,Est Ensemble (T8),...,Indicateur tarif,"1,38",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [169]:
df_verif["Nom de la commune adhérente"].value_counts() #Il y avait déjà des doublons ce qui expilque la petite différence de 58 lignes qu'il y aura à la fin

Nom de la commune adhérente
Gurcy-le-Châtel       3
Grisy-sur-Seine       3
Coulommiers           2
Marles-en-Brie        2
Bransles              2
                     ..
Saint-Leu-la-Forêt    1
Meudon                1
Saint-Prix            1
Sannois               1
Neuilly-sur-Seine     1
Name: count, Length: 1148, dtype: int64

## Pour régler les doublons on va pouvoir faire un groupby() sur les communes en gardant la valeur max de D102.0. 
### Pour une étude plus sérieuse les NaN de la colonne D102.0 pourront être remplacés par la médiane

In [170]:
df_final.columns

Index(['DPT de la commune', 'Nom de la commune adhérente', 'D102.0'], dtype='object')

In [171]:
mediane = float(df_final[df_final['D102.0'].isna() == False]['D102.0'].str.replace(",", ".").astype(float).median())

In [172]:
mediane

3.69

In [173]:
df_final['D102.0'] = df_final['D102.0'].str.replace(",", ".").astype(float)

In [174]:
df_final = df_final.fillna(mediane)

In [175]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9890 entries, 0 to 9889
Data columns (total 3 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   DPT de la commune            9890 non-null   object 
 1   Nom de la commune adhérente  9890 non-null   object 
 2   D102.0                       9890 non-null   float64
dtypes: float64(1), object(2)
memory usage: 231.9+ KB


In [176]:
df_final = df_final.groupby('Nom de la commune adhérente').max().reset_index()

In [177]:
df_final

,Nom de la commune adhérente,DPT de la commune,D102.0
0,Abbéville-la-Rivière,91,3.85
1,Ableiges,95,2.58
2,Ablis,78,2.72
3,Ablon-sur-Seine,94,3.69
4,Achères,78,3.64
...,...,...,...
1143,Étréchy,91,5.16
1144,Étrépilly,77,2.60
1145,Évecquemont,78,3.64
1146,Évry-Courcouronnes,91,3.06


# On obtient le df voulu pour l'année 2023. Dans un 2e notebook on va pouvoir créer une fonction qui va le faire automatiquement pour toutes les années disponibles

Pour l'assainissement il suffira de prendre simplement le détail et la colonne D204.0 qui correspond au prix de l'assainissement